In [2]:
import pandas as pd
import numpy as np
import re
from urllib.request import Request, urlopen
import requests
import json
from bs4 import BeautifulSoup

# Mining Weapon Data - Classic.WowHead.Com

In [2]:
url = 'https://classic.wowhead.com/weapons&xml'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'xml')
weapon_div = soup.find_all('select', id="filter-facet-type")[0]
weapon_list = weapon_div.find_all('option')
weapon_value = [w['value'] for w in weapon_list]
weapon_name = [w.text for w in weapon_list]
weapon_list = [w.text.lower().replace(' ','-').replace('(','').replace(')','') for w in weapon_list]

print('Number of weapons: {0}'.format(len(weapon_list)))
display(weapon_name)

Number of weapons: 17


['Daggers',
 'Fist Weapons',
 'One-Handed Axes',
 'One-Handed Maces',
 'One-Handed Swords',
 'Polearms',
 'Staves',
 'Two-Handed Axes',
 'Two-Handed Maces',
 'Two-Handed Swords',
 'Bows',
 'Crossbows',
 'Guns',
 'Thrown',
 'Wands',
 'Fishing Poles',
 'Miscellaneous (Weapons)']

In [3]:
weapon_df_raw = pd.DataFrame()
for weap in weapon_list:
    for bop in range(1,3):
        url = 'https://classic.wowhead.com/{0}?filter=2;{1};0'.format(weap,bop)
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(url,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page, 'xml')

        pattern = re.compile("var listviewitems = \[(.*?)\];")
        try:
            json_raw = pattern.findall(soup.text)[0]
            json_split = json_raw.split('},')
        except IndexError as e:
            continue

        for j in range(len(json_split)):
            json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
            json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
            json_split[j] = re.sub(', ', '-', json_split[j])
            json_elements = json_split[j].split(',')
            col = []
            data = []
            for ele in json_elements:
                dat = ele.split(':')
                try:
                    col.append(dat[0])
                    data.append(dat[1])
                except IndexError as e:
                    print(dat)
                    print()
                    display(json_split[j])
                    raise IndexError(e)
            tmp_df = pd.DataFrame(data=[data], columns=col)
            tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
            if bop == 2:
                tmp_df['bop'] = 0
            else:
                tmp_df['bop'] = 1
            weapon_df_raw = pd.concat([tmp_df,weapon_df_raw], axis=0, ignore_index=True, sort=False)

display(weapon_df_raw)

,classs,displayid,flags2,id,level,name,reqlevel,slot,subclass,firstseenpatch,...,holsplpwr,healthrgn,reqrace,races,natsplpwr,reqfaction,reqrep,splpen,feratkpwr,itemset
0,2,34365,8192,21713,1,7Elune's Candle,10,0,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,13853,8192,7170,20,8Twain Random Sword FOO,15,17,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,13583,8192,6174,20,8Twain Random Sword,15,17,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,6589,8192,4616,1,7Ryedol's Lucky Pick,NaN,13,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,8568,8192,5956,1,7Blacksmith Hammer,1,21,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,6568,8192,2901,4,7Mining Pick,1,21,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,7494,8192,6219,10,7Arclight Spanner,NaN,21,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,6440,8192,7005,4,7Skinning Knife,1,13,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,7867,8192,5410,1,7OLDCeremonial Club,NaN,0,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,6567,8192,4704,1,7OLDCeremonial Club,NaN,21,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# weapon_df = weapon_df_raw[[
#     'id',
#     'name',
#     'subclass',
#     'itemset',
#     'bop',
#     'sellprice',
#     'buyprice',
#     'reqfaction',
#     'classes',
#     'races',
#     'reqskill',
#     'reqskillrank',
#     'reqrep',
#     'reqlevel',
#     'level',
#     'maxcount',
#     'dmgmin1',
#     'dmgmax1',
#     'speed',
#     'dps',
#     'armor',
#     'dura',
#     'sta',
#     'spi',
#     'int',
#     'agi',
#     'str',
#     'def',
#     'atkpwr',
#     'healthrgn',
#     'manargn',
#     'rgdatkpwr',
#     'feratkpwr',
#     'splpwr',
#     'arcsplpwr',
#     'frosplpwr',
#     'shasplpwr',
#     'firsplpwr',
#     'holsplpwr',
#     'natsplpwr',
#     'splheal',
#     'firres',
#     'arcres',
#     'frores',
#     'shares',
#     'natres',
#     'parrypct',
#     'mlehitpct',
#     'rgdhitpct',
#     'splhitpct',
#     'mlecritstrkpct',
#     'rgdcritstrkpct',
#     'splcritstrkpct',
#     'splpen'
# ]]

# for i in range(len(weapon_df)):
#     weapon_df.at[i,'name'] = re.sub('^[0-9]+', '', weapon_df.loc[i,'name'])
#     weapon_df.at[i,'name'] = re.sub('-', ', ', weapon_df.loc[i,'name'])
    
# for i in range(len(weapon_value)):
#     weapon_df['subclass'] = weapon_df['subclass'].replace(weapon_value[i],weapon_name[i])
# display(weapon_df)

In [5]:
weapon_df_raw.to_csv('data/weapons_df.csv', index=False)

# Mine Armour Data

In [6]:
url = 'https://classic.wowhead.com/armor&xml'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'xml')
slot_div = soup.find_all('select', id="filter-facet-slot")[0]
slot_list = slot_div.find_all('option')
slot_value = [s['value'] for s in slot_list]
slot_name = [s.text for s in slot_list]
slot_list = [s.text.lower().replace(' ','-').replace('(','').replace(')','') for s in slot_list]

print(slot_value)
print(slot_name)

['16', '5', '8', '11', '10', '1', '23', '7', '2', '28', '14', '4', '3', '19', '12', '6', '9']
['Back', 'Chest', 'Feet', 'Finger', 'Hands', 'Head', 'Held In Off-hand', 'Legs', 'Neck', 'Relic', 'Shield', 'Shirt', 'Shoulder', 'Tabard', 'Trinket', 'Waist', 'Wrist']


In [7]:
url = 'https://classic.wowhead.com/armor/slot:23/type:4&xml'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'xml')

'Your criteria did not match any items' in str(soup)

True

In [8]:
type_div = soup.find_all('select', id="filter-facet-type")[0]
type_list = type_div.find_all('option')
type_value = [s['value'] for s in type_list]
type_name = [s.text for s in type_list]
type_list = [s.text.lower().replace(' ','-').replace('(','').replace(')','') for s in type_list]

print(type_value)
print(type_name)

['16', '5', '8', '11', '10', '1', '23', '7', '2', '28', '14', '4', '3', '19', '12', '6', '9']
['Cloth Armor', 'Leather Armor', 'Mail Armor', 'Plate Armor', 'Amulets', 'Cloaks', 'Idols', 'Librams', 'Off-hand Frills', 'Relics', 'Rings', 'Tabards', 'Trinkets', 'Shields', 'Shirts', 'Totems', 'Sigils', 'Miscellaneous (Armor)']


In [9]:
armor_df_raw = pd.DataFrame()
for s in slot_value:
    for t in type_value:
        for bop in range(1,3):
            url = 'https://classic.wowhead.com/armor/slot:{0}/type:{1}?filter=2;{2};0'.format(s, t, bop)
            hdr = {'User-Agent': 'Mozilla/5.0'}
            req = Request(url,headers=hdr)
            page = urlopen(req)
            soup = BeautifulSoup(page, 'xml')

            pattern = re.compile("var listviewitems = \[(.*?)\];")
            try:
                json_raw = pattern.findall(soup.text)[0]
                json_split = json_raw.split('},')
            except IndexError as e:
                continue

            for j in range(len(json_split)):
                json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
                json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
                json_split[j] = re.sub(', ', '-', json_split[j])
                json_elements = json_split[j].split(',')
                col = []
                data = []
                for ele in json_elements:
                    dat = ele.split(':')
                    try:
                        col.append(dat[0])
                        data.append(dat[1])
                    except IndexError as e:
                        print(dat)
                        print()
                        display(json_split[j])
                        raise IndexError(e)
                tmp_df = pd.DataFrame(data=[data], columns=col)
                tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
                if bop == 2:
                    tmp_df['bop'] = 0
                else:
                    tmp_df['bop'] = 1
                armor_df_raw = pd.concat([tmp_df,armor_df_raw], axis=0, ignore_index=True, sort=False)
                
display(armor_df_raw)

,armor,classs,displayid,flags2,id,level,name,reqlevel,slot,slotbak,...,t,ti,sourcemore,icon,s,sheathtype,dmg,q,holsplpwr,z
0,148,4,27350,8192,9972,42,6Embossed Plate Bracers,40,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,206,4,26829,8192,14965,49,6High Chief's Bindings,44,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,225,4,27406,8192,10171,54,6Templar Bracers,49,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,116,4,26646,8192,14903,40,6Saltstone Armsplints,40,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148,4,26454,8192,14941,42,6Warbringer's Armsplints,40,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,195,4,26794,8192,14914,46,6Jade Bracers,41,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,210,4,26869,8192,14923,50,6Lofty Armguards,45,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,241,4,26878,8192,14869,58,6Warleader's Bracers,53,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,287,4,35555,8192,23090,63,5Bracers of Undead Slaying,58,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,199,4,9388,8192,8083,53,8Light Plate Bracers,48,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
armor_df_raw.to_csv('data/armor_df.csv', index=False)

# Mine Containers

In [11]:
container_df_raw = pd.DataFrame()

for bop in range(1,3):
    url = 'https://classic.wowhead.com/containers?filter=2;{0};0'.format(bop)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'xml')

    pattern = re.compile("var listviewitems = \[(.*?)\];")
    try:
        json_raw = pattern.findall(soup.text)[0]
        json_split = json_raw.split('},')
    except IndexError as e:
        continue

    for j in range(len(json_split)):
        json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
        json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
        json_split[j] = re.sub(', ', '-', json_split[j])
        json_elements = json_split[j].split(',')
        col = []
        data = []
        for ele in json_elements:
            dat = ele.split(':')
            try:
                col.append(dat[0])
                data.append(dat[1])
            except IndexError as e:
                print(dat)
                print()
                display(json_split[j])
                raise IndexError(e)
        tmp_df = pd.DataFrame(data=[data], columns=col)
        tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
        if bop == 2:
            tmp_df['bop'] = 0
        else:
            tmp_df['bop'] = 1
        container_df_raw = pd.concat([tmp_df,container_df_raw], axis=0, ignore_index=True, sort=False)
    
display(container_df_raw)

,classs,displayid,flags2,id,level,name,nslots,slot,slotbak,subclass,...,popularity,bop,avgbuyout,commondrop,buyprice,reqlevel,contentPhase,reqclass,classes,side
0,1,3568,8192,1623,35,7Raptor Skin Pouch,12,18,18,0,...,12},0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3410,8192,3233,15,7Gnoll Hide Sack,8,18,18,0,...,18,0,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,8442,8192,4241,15,7Green Woolen Bag,8,18,18,0,...,19,0,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3565,8192,5573,15,7Green Leather Bag,8,18,18,0,...,20,0,815,true,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1025,8192,856,15,7Blue Leather Bag,8,18,18,0,...,21,0,750,true,NaN,NaN,NaN,NaN,NaN,NaN
5,1,1025,8192,1470,25,7Murloc Skin Bag,10,18,18,0,...,23,0,3830,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,3568,8192,5575,25,7Large Green Sack,10,18,18,0,...,26,0,3400,true,NaN,NaN,NaN,NaN,NaN,NaN
7,1,8269,8192,5574,15,7White Leather Bag,8,18,18,0,...,27,0,800,true,NaN,NaN,NaN,NaN,NaN,NaN
8,1,4056,8192,857,25,7Large Red Sack,10,18,18,0,...,28,0,3100,true,NaN,NaN,NaN,NaN,NaN,NaN
9,1,981,8192,2657,15,7Red Leather Bag,8,18,18,0,...,30,0,775,true,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
container_df_raw.to_csv('data/container_df.csv', index=False)

# Mine Consumable Data

In [13]:
consumable_df_raw = pd.DataFrame()

for bop in range(1,3):
    url = 'https://classic.wowhead.com/consumable-items?filter=2;{0};0'.format(bop)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'xml')

    pattern = re.compile("var listviewitems = \[(.*?)\];")
    try:
        json_raw = pattern.findall(soup.text)[0]
        json_split = json_raw.split('},')
    except IndexError as e:
        continue

    for j in range(len(json_split)):
        json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
        json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
        json_split[j] = re.sub(', ', '-', json_split[j])
        json_elements = json_split[j].split(',')
        col = []
        data = []
        for ele in json_elements:
            dat = ele.split(':')
            try:
                col.append(dat[0])
                data.append(dat[1])
            except IndexError as e:
                print(dat)
                print()
                display(json_split[j])
                raise IndexError(e)
        tmp_df = pd.DataFrame(data=[data], columns=col)
        tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
        if bop == 2:
            tmp_df['bop'] = 0
        else:
            tmp_df['bop'] = 1
        consumable_df_raw = pd.concat([tmp_df,consumable_df_raw], axis=0, ignore_index=True, sort=False)
        
display(consumable_df_raw)

,classs,flags2,id,level,name,slot,subclass,firstseenpatch,displayid,dmgrange,...,reqrep,splcritstrkpct,spldmg,mlehitpct,rgdhitpct,mleatkpwr,blockamount,rgdatkpwr,atkpwr,splhitpct
0,0,8192,22036,1,7QAEnchant Bracer +4 Mana\\5,0,6,0,7899,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,8192,17893,1,7QAEnchant Shield +9 Spirit,0,6,0,7899,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,8192,17892,1,7QAEnchant Shield +8 Frost Resistance,0,6,0,7899,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,8192,17828,1,7QAEnchant Bracer +9 Stamina,0,6,0,7899,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,8192,16104,1,7QAEnchhelp Cloak +7 Fire Resistance,0,6,0,7899,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,8192,22293,1,7Package of Cards,0,0,0,34745,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,8192,22169,1,7Gift of Friendship,0,0,0,34674,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,8192,22144,1,7Bluffwatcher's Card,0,0,0,34736,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,8192,22122,1,7Pledge of Loyalty,0,0,0,34755,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,8192,21559,1,7Small Green Rocket,0,0,0,34280,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
consumable_df_raw.to_csv('data/consumable_df.csv', index=False)

# Mine Trade Goods Data

In [15]:
tradegoods_df_raw = pd.DataFrame()

for bop in range(1,3):
    url = 'https://classic.wowhead.com/trade-goods?filter=2;{0};0'.format(bop)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'xml')

    pattern = re.compile("var listviewitems = \[(.*?)\];")
    try:
        json_raw = pattern.findall(soup.text)[0]
        json_split = json_raw.split('},')
    except IndexError as e:
        continue

    for j in range(len(json_split)):
        json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
        json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
        json_split[j] = re.sub(', ', '-', json_split[j])
        json_elements = json_split[j].split(',')
        col = []
        data = []
        for ele in json_elements:
            dat = ele.split(':')
            try:
                col.append(dat[0])
                data.append(dat[1])
            except IndexError as e:
                print(dat)
                print()
                display(json_split[j])
                raise IndexError(e)
        tmp_df = pd.DataFrame(data=[data], columns=col)
        tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
        if bop == 2:
            tmp_df['bop'] = 0
        else:
            tmp_df['bop'] = 1
        tradegoods_df_raw = pd.concat([tmp_df,tradegoods_df_raw], axis=0, ignore_index=True, sort=False)
        
display(tradegoods_df_raw)

,classs,flags2,id,level,name,slot,subclass,firstseenpatch,displayid,dmgrange,...,sheathtype,speed,reqspell,dd,dmgmax1,dmgmin1,dmgtype1,dps,bd,maxcount
0,7,8192,6374,20,7Enchanted Powder,0,12,0,6371,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,8192,3777,28,8Lethargy Root,0,0,0,6371,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,8192,20423,1,7Sandy Scorpid Claw,0,11,0,2627,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,8192,2929,12,7Tomb Rot,0,11,0,7414,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,8192,6216,5,7Mystical Powder,0,12,0,7414,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,8192,6987,10,7Fish Scale,0,0,0,6628,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,8192,23055,1,7Word of Thawing,0,11,0,1013,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,8192,2693,30,7OLD Stormwind Seasoning Salts,0,11,0,1443,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,8192,2932,40,7Torment Vine,0,11,0,7415,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,7,8192,10719,41,7Mobile Alarm,0,3,0,16536,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
tradegoods_df_raw.to_csv('data/tradegoods_df.csv', index=False)

# Mine Projectile Data

In [17]:
projectile_df_raw = pd.DataFrame()

for bop in range(1,3):
    url = 'https://classic.wowhead.com/projectile-items?filter=2;{0};0'.format(bop)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'xml')

    pattern = re.compile("var listviewitems = \[(.*?)\];")
    try:
        json_raw = pattern.findall(soup.text)[0]
        json_split = json_raw.split('},')
    except IndexError as e:
        continue

    for j in range(len(json_split)):
        json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
        json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
        json_split[j] = re.sub(', ', '-', json_split[j])
        json_elements = json_split[j].split(',')
        col = []
        data = []
        for ele in json_elements:
            dat = ele.split(':')
            try:
                col.append(dat[0])
                data.append(dat[1])
            except IndexError as e:
                print(dat)
                print()
                display(json_split[j])
                raise IndexError(e)
        tmp_df = pd.DataFrame(data=[data], columns=col)
        tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
        if bop == 2:
            tmp_df['bop'] = 0
        else:
            tmp_df['bop'] = 1
        projectile_df_raw = pd.concat([tmp_df,projectile_df_raw], axis=0, ignore_index=True, sort=False)
        
display(projectile_df_raw)

,classs,displayid,dps,flags2,id,level,name,slot,slotbak,subclass,...,dmgrange,dmgtype1,reqlevel,speed,popularity,bop,avgbuyout,sellprice,contentPhase,buyprice
0,6,23211,1.00,8192,19082,1,7Monster - Fire Arrow,24,24,2,...,1.00,0,1,3.00,12},0,NaN,NaN,NaN,NaN
1,6,5996,2.00,8192,2514,8,7Depricated Sharp Arrow,24,24,2,...,1.00,0,3,3.00,14,0,NaN,NaN,NaN,NaN
2,6,5998,2.00,8192,4960,7,7Flash Pellet,24,24,3,...,1.00,0,1,3.00,32,0,8,12,NaN,NaN
3,6,5998,4.50,8192,8068,20,7Crafted Heavy Shot,24,24,3,...,1.00,0,15,3.00,67,0,2,NaN,NaN,NaN
4,6,5996,8.50,8192,3031,35,7Depricated Razor Arrow,24,24,2,...,1.00,0,30,3.00,68,0,NaN,NaN,NaN,NaN
5,6,5998,8.50,8192,8069,35,7Crafted Solid Shot,24,24,3,...,1.00,0,30,3.00,69,0,4,NaN,NaN,NaN
6,6,5998,2.00,8192,8067,10,7Crafted Light Shot,24,24,3,...,1.00,0,5,3.00,70,0,NaN,NaN,NaN,NaN
7,6,5998,4.00,8192,5568,18,7Smooth Pebble,24,24,3,...,1.00,0,13,3.00,70,0,4,4,NaN,NaN
8,6,2418,18.00,8192,11630,52,5Rockshard Pellets,24,24,3,...,1.00,0,47,3.00,82,0,75,5,1,NaN
9,6,26498,11.50,8192,9399,40,6Precision Arrow,24,24,2,...,1.00,0,35,3.00,118,0,16,3,NaN,NaN


In [18]:
projectile_df_raw.to_csv('data/projectile_df.csv', index=False)

# Mine Quiver Data

In [19]:
quiver_df_raw = pd.DataFrame()

for bop in range(1,3):
    url = 'https://classic.wowhead.com/quiver-items?filter=2;{0};0'.format(bop)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'xml')

    pattern = re.compile("var listviewitems = \[(.*?)\];")
    try:
        json_raw = pattern.findall(soup.text)[0]
        json_split = json_raw.split('},')
    except IndexError as e:
        continue

    for j in range(len(json_split)):
        json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
        json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
        json_split[j] = re.sub(', ', '-', json_split[j])
        json_elements = json_split[j].split(',')
        col = []
        data = []
        for ele in json_elements:
            dat = ele.split(':')
            try:
                col.append(dat[0])
                data.append(dat[1])
            except IndexError as e:
                print(dat)
                print()
                display(json_split[j])
                raise IndexError(e)
        tmp_df = pd.DataFrame(data=[data], columns=col)
        tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
        if bop == 2:
            tmp_df['bop'] = 0
        else:
            tmp_df['bop'] = 1
        quiver_df_raw = pd.concat([tmp_df,quiver_df_raw], axis=0, ignore_index=True, sort=False)
        
display(quiver_df_raw)

,classs,displayid,flags2,id,level,name,nslots,slot,slotbak,subclass,...,dmgrange,reqlevel,sellprice,popularity,bop,side,maxcount,reqclass,classes,contentPhase
0,11,21318,8192,5439,5,7Small Quiver,8,18,18,2,...,1.00,1,25,64},0,NaN,NaN,NaN,NaN,NaN
1,11,21328,8192,2101,1,7Light Quiver,6,18,18,2,...,1.00,1,1,76,0,NaN,NaN,NaN,NaN,NaN
2,11,1816,8192,5441,5,7Small Shot Pouch,8,18,18,3,...,1.00,1,250,143,0,NaN,NaN,NaN,NaN,NaN
3,11,1816,8192,7279,5,7Small Leather Ammo Pouch,8,18,18,3,...,1.00,1,25,161,0,NaN,NaN,NaN,NaN,NaN
4,11,21330,8192,7278,5,7Light Leather Quiver,8,18,18,2,...,1.00,1,25,161,0,NaN,NaN,NaN,NaN,NaN
5,11,1816,8192,2102,1,7Small Ammo Pouch,6,18,18,3,...,1.00,1,1,176,0,NaN,NaN,NaN,NaN,NaN
6,11,2585,8192,7372,35,6Heavy Leather Ammo Pouch,14,18,18,3,...,1.00,30,500,204,0,NaN,NaN,NaN,NaN,NaN
7,11,21329,8192,11362,15,7Medium Quiver,10,18,18,2,...,1.00,10,250,268,0,NaN,NaN,NaN,NaN,NaN
8,11,21322,8192,7371,35,6Heavy Quiver,14,18,18,2,...,1.00,30,500,320,0,NaN,NaN,NaN,NaN,NaN
9,11,1816,8192,11363,15,7Medium Shot Pouch,10,18,18,3,...,1.00,10,250,328,0,NaN,NaN,NaN,NaN,NaN


In [20]:
quiver_df_raw.to_csv('data/quiver_df.csv', index=False)

# Mine Recipe Data

In [4]:
url = 'https://classic.wowhead.com/recipe-items&xml'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, 'xml')
type_div = soup.find_all('select', id="filter-facet-type")[0]
type_list = type_div.find_all('option')
type_value = [s['value'] for s in type_list]
type_name = [s.text for s in type_list]
type_list = [s.text.lower().replace(' ','-').replace('(','').replace(')','') for s in type_list]

print(type_value)
print(type_name)

['0', '6', '4', '5', '8', '3', '7', '9', '1', '12', '2']
['Books', 'Alchemy Recipes', 'Blacksmithing Plans', 'Cooking Recipes', 'Enchanting Formulae', 'Engineering Schematics', 'First Aid Books', 'Fishing Books', 'Leatherworking Patterns', 'Mining Guides', 'Tailoring Patterns']


In [7]:
recipe_df_raw = pd.DataFrame()

for t in type_value:
    for bop in range(1,3):
        url = 'https://classic.wowhead.com/recipe-items?filter=2;{0};0'.format(bop)
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(url,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page, 'xml')

        pattern = re.compile("var listviewitems = \[(.*?)\];")
        try:
            json_raw = pattern.findall(soup.text)[0]
            json_split = json_raw.split('},')
        except IndexError as e:
            continue

        for j in range(len(json_split)):
            json_split[j] = re.sub('"sourcemore"\:\[(.*?)\],', '',json_split[j]).replace('{','').replace('"','')
            json_split[j] = re.sub('source:\[(.*?)\],', '', json_split[j])
            json_split[j] = re.sub(', ', '-', json_split[j])
            json_elements = json_split[j].split(',')
            col = []
            data = []
            for ele in json_elements:
                dat = ele.split(':')
                
                if len(dat) == 3:
                    dat[1] += ':' + dat[2]
                    
                try:
                    col.append(dat[0])
                    data.append(dat[1])
                except IndexError as e:
                    print(dat)
                    print()
                    display(json_split[j])
                    raise IndexError(e)
            tmp_df = pd.DataFrame(data=[data], columns=col)
            tmp_df = tmp_df.loc[:,~tmp_df.columns.duplicated()]
            if bop == 2:
                tmp_df['bop'] = 0
            else:
                tmp_df['bop'] = 1
            recipe_df_raw = pd.concat([tmp_df,recipe_df_raw], axis=0, ignore_index=True, sort=False)
        
display(recipe_df_raw)

,classs,flags2,id,level,name,reqclass,reqlevel,slot,subclass,firstseenpatch,...,z,sourcemore,c2,reqfaction,reqrep,c,dd,maxcount,bd,side
0,9,8192,8919,32,7Libram: Holy Strike V,2,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,8192,8828,32,7Tome of Frostbolt VI,128,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,8192,8826,32,7Tome of Arcane Missiles IV,128,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,8192,5671,32,7Libram: Seal of Might III,2,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8192,4287,32,7Codex of Nullify Disease II,16,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9,8192,4175,32,7Tablet of Lightning Shield IV,64,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,9,8192,3092,32,7Tome of Flamestrike III,128,32,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,8192,9092,34,7Tablet of Sentry Totem,64,34,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,8192,8771,34,7Book of Rejuvenation VI,1024,34,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,8192,8770,34,7Book of Moonfire VI,1024,34,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
recipe_df_raw.to_csv('data/recipe_df.csv', index=False)